In [1]:
import pandas as pd
from cleantext import clean
import re
import torch.nn.functional as F


import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time
import math

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/remi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
reqres = pd.read_json("/home/remi/Downloads/Requêtes et résultats attendus.json")

In [21]:
reqres

,question,expectedResults
0,"Je suis tombé malade pendant mes congés, est-c...",[{'slug': 'contribution/si-le-salarie-est-mala...
1,"J’ai été en arrêt de travail pendant 45 jours,...",[{'slug': 'information/suivi-medical-et-accomp...
2,J’ai quitté mon entreprise il y a 7 jours et j...,[{'slug': 'modeles-de-courriers/lettre-de-recl...
3,Quel est mon salaire pendant mes congés ?,[{'slug': 'fiche-service-public/comment-est-ca...
4,"Je suis licencié pour des raisons économiques,...",[{'slug': 'fiche-service-public/licenciement-e...
5,Comment calculer l’ancienneté dans l’entrepris...,[{'slug': 'fiche-service-public/comment-calcul...
6,A combien de jours de congés payés ai-je le dr...,"[{'slug': 'fiche-service-public/conges-payes',..."
7,Si je démissionne et que je suis enceinte est-...,[{'slug': 'fiche-service-public/une-salariee-p...
8,Ai-je le droit à une indemnité si je suis décl...,[{'slug': 'fiche-service-public/le-salarie-tou...
9,J'ai 62 ans. Mon employeur peut-il m'imposer d...,[{'slug': 'fiche-service-public/un-employeur-p...


In [22]:
window_size = 310
slide = 70

In [23]:
def make_windows(content, w_size, slide):
    if slide > w_size :
        raise Exception()
    tokens = content.split(" ")
    l = len(tokens)    
       
    n = math.ceil(l / (w_size-slide))        

    if n==1 :
        return [content]
    else :
        windows = []
        for i in range(n) :            
            x = i*slide
            if x < l :
                y = x + w_size
                w = tokens[x:y]
                windows.append(" ".join(w))
        
        return windows

In [3]:
fiches = pd.read_csv("/home/remi/Downloads/xp-documents-fiches (2).csv")

In [4]:
fiches["cleaned_text"] = fiches["text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
fiches["tokens"] = fiches["cleaned_text"].apply(lambda t : len(str(t).split(" ")))

In [5]:
fiches = fiches[fiches["tokens"]<3000]

In [24]:
fiches["windows"] = fiches["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
fiches.drop(columns=["cleaned_text", "text"], inplace=True)
fiches = fiches.explode("windows", ignore_index=True)
fiches.rename(columns={"windows": "text"}, inplace=True)

In [25]:
fiches["cdtnId-fiches"] = fiches["cdtnId"]
fiches["cdtnId"] = fiches["cdtnId"].apply(lambda x : x.split("-")[0])

In [26]:
contribs = pd.read_csv("/home/remi/Downloads/xp-documents-contribs (1).csv")

In [27]:
contribs.drop_duplicates(subset=["answers.generic.text"], inplace=True)
contribs["cleaned_text"] = contribs["answers.generic.text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
contribs["tokens"] = contribs["cleaned_text"].apply(lambda t : len(str(t).split(" ")))
contribs["windows"] = contribs["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
contribs.drop(columns=["cleaned_text","answers.generic.text","text"], inplace=True)
contribs = contribs.explode("windows", ignore_index=True)
contribs.rename(columns={"windows": "text"}, inplace=True)

In [28]:
contribs["slug"] = contribs["slug"].apply(lambda s: "-".join(s.split("-")[1:]))

In [29]:
contribs_tensor = torch.load('/home/remi/Desktop/contribs_tensor3.pt')
fiches_tensor = torch.load('/home/remi/Desktop/fiches_tensor3.pt')
tensor_docs = torch.vstack([fiches_tensor, contribs_tensor])

In [30]:
passages_set = pd.concat([fiches, contribs]).reset_index()
passages_set.shape

(4795, 9)

In [31]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

In [32]:
def search(query, verbose=False):
    # embbed query
    batch_dict = tokenizer('query: '+ query, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).detach()
    question_normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

    # score against all passages and retrieve top k
    
    retrieval_scores = (question_normalized_embeddings @ tensor_docs.T)[0] * 100
    k = 20
    
    j = torch.argmax(retrieval_scores).item()
    matches = torch.topk(retrieval_scores, k).values.tolist()
    matching_indices = torch.topk(retrieval_scores, k).indices.tolist()
    #print(matches)
    #print(matching_indices)

    # reranker
    batch = ['query: '+ query]

    for m in matching_indices :    
        batch.append('passage: ' + passages_set.loc[m]["text"])
    
    #print(batch)
        
    batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).detach()
    normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

    reranker_scores = (normalized_embeddings[0] @ normalized_embeddings[1:].T) * 100
    #print(reranker_scores)
        
    #print(reranker_scores)
    rr_matches = torch.topk(reranker_scores, k).values.tolist()
    rr_matching_indices = torch.topk(reranker_scores, k).indices.tolist()
    #print(len(rr_matching_indices))
    #print(rr_matching_indices)

    final_indices = [matching_indices[m] for m in rr_matching_indices]
    #print(final_indices)
    
    #print(query)

    res = passages_set.loc[final_indices][["index","source","cdtnId","title","text", "tokens", "slug"]]
    
    # res = res.head(7)

    if verbose:
        display(res[["source", "cdtnId", "slug", "title", "text"]].style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
        }))

    return {"dot_prod": matching_indices, "reranked": final_indices}

In [30]:
indices = search("J’ai quitté mon entreprise il y a 7 jours et je n’ai toujours pas eu de certificat de travail, que dois-je faire ?")

In [28]:
i = 21
q = reqres.loc[i].question
exp = reqres.loc[i].expectedResults

print(q)
indices = search(q)

Combien de temps dure le maintien de salaire en arrêt maladie ?


KeyboardInterrupt: 

In [96]:
exp

[{'slug': 'contribution/les-conges-pour-evenements-familiaux',
  'cdtnId': 'eba7a4592f'},
 {'slug': 'fiche-service-public/conge-du-salarie-pour-mariage-ou-pacs',
  'cdtnId': '43c5c7bc41'}]

In [30]:
rr_res = passages_set.loc[indices["reranked"]][["index","source", "slug", "cdtnId","title","text", "tokens"]]
rr_res

,index,source,slug,cdtnId,title,text,tokens
3401,3401,fiches_service_public,certificat-de-travail,59a4d4173c,Certificat de travail,lorsque le contrat de travail du salarié prend...,503
786,786,fiches_ministere_travail,etre-recrute-en-emploi-franc#Quels-sont-les-do...,4587639f3a-4011,Être recruté en emploi franc : quels sont les ...,vous devrez fournir à votre employeur :une att...,82
3403,3403,fiches_service_public,certificat-de-travail,59a4d4173c,Certificat de travail,"par exemple, faire valoir ses droits à une méd...",503
3451,3451,fiches_service_public,accident-du-travail-reprise-du-travail-du-salarie,5f23027d1e,Accident du travail : reprise du travail du sa...,"de travail léger, qui est indemnisé sous condi...",1852
256,256,fiches_ministere_travail,les-documents-remis-aux-salaries-lors-de-la-ru...,1e4c116fcb-1281,Les documents remis aux salariés lors de la ru...,l'employeur est tenu de délivrer le certificat...,114
3402,3402,fiches_service_public,certificat-de-travail,59a4d4173c,Certificat de travail,la réglementation. qu'est-ce que le certificat...,503
2850,2850,fiches_service_public,arret-maladie-reprise-du-travail-du-salarie,2294993de1,Arrêt maladie : reprise du travail du salarié,"à l'initiative de votre employeur, au plus tar...",2249
4464,4464,fiches_service_public,fin-de-contrat-documents-a-remettre-au-salarie,dd8e1cab52,Fin de contrat : documents à remettre au salarié,à l'occasion de la rupture du contrat de trava...,316
4115,4115,fiches_service_public,maladie-professionnelle-reprise-du-travail-du-...,b26aa16ef3,Maladie professionnelle : reprise du travail d...,pendant les heures de travail . votre absence ...,1288
4113,4113,fiches_service_public,maladie-professionnelle-reprise-du-travail-du-...,b26aa16ef3,Maladie professionnelle : reprise du travail d...,de reprise du travail ? vous devez passer une ...,1288


In [69]:
def process(e, verbose = False) :
    q = e.question
    exp = e.expectedResults

    print(i)
    print(q)
    indices = search(q)
    
    
    dp_res = passages_set.loc[indices["dot_prod"]][["index","source", "slug", "cdtnId","title","text", "tokens"]].reset_index()
    if verbose:
        display(dp_res[["source", "cdtnId", "slug", "title", "text"]].style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
        }))
    
    rr_res = passages_set.loc[indices["reranked"]][["index","source", "slug", "cdtnId","title","text", "tokens"]].reset_index()
    if verbose:
        display(rr_res[["source", "cdtnId", "slug", "title", "text"]].style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
        }))

    dp_res.drop_duplicates(subset=['cdtnId'], inplace=True)
    rr_res.drop_duplicates(subset=['cdtnId'], inplace=True)
    
    for e in exp :
        cdtnId = e["cdtnId"]    
        slug = e["slug"].split("/")[1]
        
        found = passages_set[passages_set["cdtnId"] == cdtnId].shape[0] > 0
    
        if not found :        
            found = passages_set[passages_set["slug"] == slug].shape[0] > 0

        if not found :
             e["dp_pos"] = -1
             e["rr_pos"] = -1

        e["exists"] = found
    
        if found :
            dp_pos = dp_res[dp_res["cdtnId"] == cdtnId]
            if dp_pos.shape[0] < 1 :
                dp_pos = dp_res[dp_res["slug"] == slug]
            if dp_pos.shape[0] > 0 :
                e["dp_pos"] = dp_pos.index[0]
            else :
                e["dp_pos"] = -1
    
            rr_pos = rr_res[rr_res["cdtnId"] == cdtnId]
            if rr_pos.shape[0] < 1 :
                rr_pos = rr_res[rr_res["slug"] == slug]
            if rr_pos.shape[0] > 0 :
                e["rr_pos"] = rr_pos.index[0]
            else :
                e["rr_pos"] = -1
    df = pd.json_normalize(exp)
    display(df[["slug","cdtnId","exists","dp_pos","rr_pos"]].style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
    }))
    
    return df


In [87]:
results = []
for i in range(len(reqres)) :    
    results.append(process(reqres.iloc[i]))
    print("\n")

0
Je suis tombé malade pendant mes congés, est-ce que mes congés peuvent être repoussés ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences,8dffdbc6b9,True,9,8
1,fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie,6e888b3f6e,True,6,3
2,fiche-ministere-travail/les-conges-payes,fcf7934672,True,0,1




1
J’ai été en arrêt de travail pendant 45 jours, dois-je faire une visite médicale avant de reprendre ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,information/suivi-medical-et-accompagnement-de-certains-salaries,3d5005b8db,False,-1,-1
1,fiche-service-public/un-salarie-doit-il-passer-une-visite-medicale-apres-un-arret-de-travail,1096b8d2c5,True,2,2
2,fiche-service-public/accident-du-travail-reprise-du-travail-du-salarie,5f23027d1e,True,3,7
3,fiche-service-public/medecine-du-travail-pour-un-salarie-du-secteur-prive,e28ccf7d6a,True,-1,-1




2
J’ai quitté mon entreprise il y a 7 jours et je n’ai toujours pas eu de certificat de travail, que dois-je faire ?


...

,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/allocation-journaliere-de-presence-parentale-ajpp,1838dc2585,True,1,0




54
Je suis travailleur handicapé. Qu'est-ce-que ça veut dire pour ma retraite ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/retraite-du-salarie-handicape,be7358fa1e,True,5,4




55



KeyError: "None of [Index(['slug', 'cdtnId', 'exists', 'dp_pos', 'rr_pos'], dtype='object')] are in the [columns]"

In [110]:
def evaluate(q, r):
    # number and percentage of available results (exists)
    existing = r[r["exists"]]
    n = existing.shape[0]
    n_ratio = n / r.shape[0]
    # for existing, % of matches
    dp = existing[existing["dp_pos"] >= 0]
    if n>0 :
        match_ratio = dp.shape[0] / n
    else :
        match_ratio = 0
        
    if match_ratio >= 0 and dp["dp_pos"].sum() :
        # mean rank for dp / rr
        # sum of dp / rr
        dp_sum = dp["dp_pos"].sum()
        dp_mean = dp_sum / dp.shape[0] 
    
        rr = existing[existing["rr_pos"] >= 0]
        rr_sum = rr["rr_pos"].sum()
        rr_mean = rr_sum / rr.shape[0]
    else : 
        dp_sum = -1
        dp_mean = -1
        rr_sum = -1
        rr_mean = -1
    
    return {"query": q, "exists":n, "exists_ratio":n_ratio, "match_ratio":match_ratio, "dp_sum":dp_sum, "dp_mean":dp_mean, "rr_sum":rr_sum, "rr_mean":rr_mean}

evaluations = []
for req, r in zip(reqres.values.tolist(), results):
    display(r[["slug","cdtnId","exists","dp_pos","rr_pos"]].style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
    }))
    evaluations.append(evaluate(req[0], r))
    

,slug,cdtnId,exists,dp_pos,rr_pos
0,contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences,8dffdbc6b9,True,9,8
1,fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie,6e888b3f6e,True,6,3
2,fiche-ministere-travail/les-conges-payes,fcf7934672,True,0,1


,slug,cdtnId,exists,dp_pos,rr_pos
0,information/suivi-medical-et-accompagnement-de-certains-salaries,3d5005b8db,False,-1,-1
1,fiche-service-public/un-salarie-doit-il-passer-une-visite-medicale-apres-un-arret-de-travail,1096b8d2c5,True,2,2
2,fiche-service-public/accident-du-travail-reprise-du-travail-du-salarie,5f23027d1e,True,3,7
3,fiche-service-public/medecine-du-travail-pour-un-salarie-du-secteur-prive,e28ccf7d6a,True,-1,-1


,slug,cdtnId,exists,dp_pos,rr_pos
0,modeles-de-courriers/lettre-de-reclamation-des-documents-de-fin-de-contrat,2564772bc9,False,-1,-1
1,fiche-service-public/certificat-de-travail,59a4d4173c,True,1,0


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/comment-est-calculee-lindemnite-de-conges-payes-du-salarie,90d12c615f,True,17,1
1,fiche-service-public/conges-payes,a5ca4ecbb9,True,1,10
2,fiche-service-public/comment-est-calculee-lindemnite-de-conges-payes-du-salarie,90d12c615f,True,17,1


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/licenciement-economique-quelles-indemnites-peut-percevoir-un-salarie,76e2ba5c40,True,8,11
1,outils/indemnite-licenciement,d8a3605790,False,-1,-1


...

,slug,cdtnId,exists,dp_pos,rr_pos
0,contribution/travail-du-dimanche-quelle-contrepartie,d094601132,True,-1,-1
1,fiche-service-public/travail-le-dimanche-dun-salarie-du-secteur-prive,1e9dda80aa,True,1,1


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/cheque-emploi-associatif-cea,a708246e55,True,0,0


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/dans-quels-cas-recourir-a-linspecteur-du-travail,e3f335210b,True,-1,-1
1,fiche-service-public/que-risque-une-entreprise-en-cas-de-travail-illegal,f8a3838c44,True,5,4
2,fiche-ministere-travail/quest-ce-que-le-travail-illegal,8ea426e6e5,True,-1,-1
3,fiche-ministere-travail/les-obligations-de-lemployeur-lors-de-lembauche,799dc7e99a,True,-1,-1


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/allocation-journaliere-de-presence-parentale-ajpp,1838dc2585,True,1,0


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/retraite-du-salarie-handicape,be7358fa1e,True,5,4


In [113]:
pd.json_normalize(evaluations).describe()

,exists,exists_ratio,match_ratio,dp_sum,dp_mean,rr_sum,rr_mean
count,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000
mean,1.963636,0.880303,0.710303,4.545455,2.201515,2.909091,1.389394
std,1.332828,0.250393,0.403119,7.981038,4.047166,5.812734,3.471002
min,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,1.000000,0.500000,-1.000000,-1.000000,-1.000000,-1.000000
50%,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,2.000000,1.000000,1.000000,7.500000,4.000000,4.000000,2.125000
max,5.000000,1.000000,1.000000,35.000000,18.000000,29.000000,14.500000


In [76]:
resp = search("bébé")
rr_res = passages_set.loc[resp["reranked"]][["index","source", "slug", "cdtnId","title","text", "tokens"]].reset_index()
display(rr_res[["source", "cdtnId", "slug", "title", "text"]].style.set_properties(**{
    'text-align': 'left',
    'white-space': 'pre-wrap',
}))

,source,cdtnId,slug,title,text
0,fiches_service_public,a38efb9268,contrat-de-travail-et-formalites-dembauche-de-lassistante-maternelle,Contrat de travail et formalités d'embauche de l'assistante maternelle,"le jour de l'embauche, le particulier employeur et l'assistant maternel concluent un contrat de travail écrit pour chaque enfant accueilli, même s'il s'agit d'enfants de la même famille. en cas d'accueil d'un autre enfant d'une même famille dont le contrat est en cours, l' ancienneté de l'assistante maternelle est reprise. cette reprise ne concerne pas les autres droits (par exemple : congés payés). toute modification du contrat de travail doit être faite par un avenant écrit. il est établi en 2 exemplaires datés, paraphés et signés par l'employeur et l'assistante maternelle. chaque partie conservera 1 exemplaire du contrat et de ses avenants. éléments obligatoires du contrat de travail informations administratives et conventionnelles type de contrat de travail identité des parties numéro d'identification employeur numéro de sécurité sociale du salarié nom de l'enfant et date de naissance date d'embauche lieu de travail et d'accueil de l'enfant références de l'agrément assurance responsabilité civile professionnelle du salarié assurance automobile (s'il y a lieu) durée de la période d'essai périodes d'accueil et horaires absences prévues de l'enfant éléments de rémunération et dates de paiement dates habituelles de congés annuels jours fériés travaillés jour de repos hebdomadaire consignes et informations concernant l'enfant santé régime alimentaire médecin de référence soins ou médicaments consignes en cas d'urgence documents à joindre au contrat de travail autorisation concernant les modes de déplacement de l'enfant documents concernant la santé de l'enfant (bulletin de vaccination...) liste des personnes titulaires de l'exercice de l'autorité parentale, information écrite en cas d'éventuel droit de visite restreint liste et coordonnées des personnes autorisées à venir chercher l'enfant au domicile de l'assistante maternelle liste des personnes à contacter en cas d'urgence et en l'absence des parents à savoir des conditions particulières, telles que par exemple la liste de aliments que l'assistante maternelle ne peut pas donner à l'enfant, peuvent être prévues au contrat. période d'essai une"
1,fiches_ministere_travail,39f928589e,ecoles-de-la-deuxieme-chance-e2c,Écoles de la deuxième chance (E2C),infographie
2,fiches_ministere_travail,a9780cc0e6,compte-personnel-de-formation-cpf,Compte personnel de formation (CPF),infographie
3,fiches_service_public,a38efb9268,contrat-de-travail-et-formalites-dembauche-de-lassistante-maternelle,Contrat de travail et formalités d'embauche de l'assistante maternelle,"être faite par un avenant écrit. il est établi en 2 exemplaires datés, paraphés et signés par l'employeur et l'assistante maternelle. chaque partie conservera 1 exemplaire du contrat et de ses avenants. éléments obligatoires du contrat de travail informations administratives et conventionnelles type de contrat de travail identité des parties numéro d'identification employeur numéro de sécurité sociale du salarié nom de l'enfant et date de naissance date d'embauche lieu de travail et d'accueil de l'enfant références de l'agrément assurance responsabilité civile professionnelle du salarié assurance automobile (s'il y a lieu) durée de la période d'essai périodes d'accueil et horaires absences prévues de l'enfant éléments de rémunération et dates de paiement dates habituelles de congés annuels jours fériés travaillés jour de repos hebdomadaire consignes et informations concernant l'enfant santé régime alimentaire médecin de référence soins ou médicaments consignes en cas d'urgence documents à joindre au contrat de travail autorisation concernant les modes de déplacement de l'enfant documents concernant la santé de l'enfant (bulletin de vaccination...) liste des personnes titulaires de l'exercice de l'autorité parentale, information

In [38]:
for i in range(len(reqres)) :    
    process(i)
    print("\n")

0
Je suis tombé malade pendant mes congés, est-ce que mes congés peuvent être repoussés ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences,8dffdbc6b9,True,9,8
1,fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie,6e888b3f6e,True,6,3
2,fiche-ministere-travail/les-conges-payes,fcf7934672,True,0,1




1
J’ai été en arrêt de travail pendant 45 jours, dois-je faire une visite médicale avant de reprendre ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,information/suivi-medical-et-accompagnement-de-certains-salaries,3d5005b8db,False,n/a,n/a
1,fiche-service-public/un-salarie-doit-il-passer-une-visite-medicale-apres-un-arret-de-travail,1096b8d2c5,True,2,2
2,fiche-service-public/accident-du-travail-reprise-du-travail-du-salarie,5f23027d1e,True,3,7
3,fiche-service-public/medecine-du-travail-pour-un-salarie-du-secteur-prive,e28ccf7d6a,True,n/a,n/a




2
J’ai quitté mon entreprise il y a 7 jours et je n’ai toujours pas eu de certificat de travail, que dois-je faire ?


...

,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/allocation-journaliere-de-presence-parentale-ajpp,1838dc2585,True,1,0




54
Je suis travailleur handicapé. Qu'est-ce-que ça veut dire pour ma retraite ?


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/retraite-du-salarie-handicape,be7358fa1e,True,5,4




55



KeyError: "None of [Index(['slug', 'cdtnId', 'exists', 'dp_pos', 'rr_pos'], dtype='object')] are in the [columns]"

In [47]:
process(3, True)

3
Quel est mon salaire pendant mes congés ?


,source,cdtnId,slug,title,text
0,fiches_service_public,dd59ddcb89,comment-beneficier-dun-conge-sans-solde,Comment bénéficier d'un congé sans solde ?,"de quoi s'agit-il ? le congé sans solde est un congé non rémunéré. vous pouvez prendre ce congé pour convenances personnelles. vous pouvez également prendre ce congé pour des besoins professionnels (créer une entreprise, par exemple). demande de congé ce congé n'est pas prévu par la législation . votre employeur n'est pas obligé de vous accorder ce congé sans solde. si la convention collective ou un accord collectif prévoit des dispositions relatives au congé sans solde, votre employeur doit les respecter. en l'absence de dispositions conventionnelles ou collectives, l'accord de votre employeur est nécessaire. il est préférable de rédiger un document écrit, qui servira de preuve en cas de litige. ce document peut prévoir les conditions suivantes : dates envisagées de début et de fin du congé possibilité de renouveler le congé conditions de retour du salarié dans l'entreprise pendant le congé votre contrat de travail est suspendu durant votre congé. vous êtes libre de consacrer votre temps de congé aux activités de son choix (activités personnelles ou professionnelles, éventuellement pour un autre employeur, sauf clause de non-concurrence ). pendant le congé sans solde, vous ne percevez pas de salaire. toutefois, votre congé sans solde peut être rémunéré dans le cadre du compte épargne-temps (cet) , sous conditions. à la fin de votre congé sans solde à la fin de votre congé, vous retrouvez votre précédent emploi ou un emploi similaire."
1,fiches_service_public,a5ca4ecbb9,conges-payes,Congés payés,"avez-vous droit à des congés payés pendant votre contrat de travail ? quelle est la durée de vos congés payés ? quand pouvez-vous prendre vos congés payés ? qui fixe la date de départ des congés payés ? pouvez-vous prendre vos congés payés en plusieurs fois ? comment êtes-vous payé pendant vos congés payés ? nous vous expliquons vos droits et vos obligations. qui a droit aux congés payés ? oui, tout salarié a droit chaque année à des congés payés. vous bénéficiez des congés payés quel que soit votre contrat de travail ( cdi , cdd ou contrat d'intérim ). vous bénéficiez des congés payés que vous travailliez à temps plein ou à temps partiel . vous bénéficiez des congés payés dès votre embauche dans l'entreprise , quelle que soit votre ancienneté, si vous obtenez l'accord de votre employeur. comment savoir à combien de jours de congés payés vous avez droit ? que vous travaillez à temps plein ou à temps partiel, vous bénéficiez de 2,5 jours ouvrables par mois de travail effectif chez le même employeur. cela correspond à 30 jours ouvrables (5 semaines) pour une année complète de travail. certaines absences sont prises en compte pour le calcul des jours de congés. lorsque le nombre de jours de congés acquis n'est pas un nombre entier, la durée du congé est portée au nombre entier immédiatement supérieur. exemple vous avez travaillé 5 mois au cours de l'année, vous bénéficiez d'un congé de 2,5 × 5 = 12,5, arrondi à 13 jours ouvrables. votre employeur peut calculer les jours de congés en jours ouvrés . dans ce cas, ce mode de calcul doit vous garantir des droits à congés au moins égaux à ceux calculés en jours ouvrables. quelle est la période prise en compte pour déterminer le nombre de jours de congés payés ? pour l'acquisition des congés"
2,fiches_service_public,bce90c20a4,quelle-est-lincidence-sur-les-droits-a-conges-payes-dun-salarie-a-temps-partiel,Quelle est l'incidence sur les droits à congés payés d'un salarié à temps partiel ?,"le principe d'égalité entre les salariés travaillant à temps plein et les salariés travaillant à temps partiel. les salariés travaillant à temps partiel ne doivent pas avoir plus de congés que les salariés travaillant à temps plein. pour décompter le nombre de jours de congés pris en tant que salarié travaillant à temps partiel, votre employeur prend en compte le

,source,cdtnId,slug,title,text
0,fiches_service_public,daf560f481,indemnite-compensatrice-de-conges-payes,Indemnité compensatrice de congés payés,"d'heures travaillées dans le mois est fixé à 147 heures (21 x 7) et le nombre d'heures non travaillées en raison des congés payés à 70 heures (10 x 7). le calcul est le suivant : 1 820 x (7x10) / (7x21) = 866,66 € le calcul du 1/10 e de la rémunération brute est effectué ainsi : (21 840/10) = 2 184 € pour un congé d'une durée de 30 jours ouvrables (ou 25 jours ouvrés ). pour une fraction de 2 semaines de congés, le calcul est le suivant : en jours ouvrables (soit 12 jours) : (21 840/10) x (12/30) = 873,60 € en jours ouvrés (soit 10 jours) : (21 840/10) x (10/25) = 873,60 € le salarié perçoit alors le montant le plus favorable, soit 873,60 € pour ses 2 semaines de congés payés. est-ce que toutes les sommes sont prises en compte dans le calcul de l'indemnité compensatrice de congés payés ? certaines sommes sont prises en compte , d'autres non pour déterminer l'indemnité compensatrice de congés payés. sommes prises en compte ou non somme prise en compte salaire de base oui majoration de salaire ( heures supplémentaires , travail de nuit , etc.) oui salaire reconstitué pendant des périodes assimilées à du travail effectif ( congé maternité , congé de paternité et d'accueil de l'enfant ou arrêt de travail pour accident de travail ou maladie professionnelle par exemple) oui indemnité de congés payés de l'année précédente oui prime d'ancienneté (si elle n'est pas versée pour l'année, périodes de travail et de congés confondues) oui prime d'assiduité versée mensuellement oui prime d'astreinte oui commissions pour les commerciaux oui prime d'expatriation oui avantages en nature oui prime de fin d'année non prime d'intéressement non prime de bilan non prime de participation non frais professionnels non 13 e mois (si la somme n'est pas versée pour"
1,fiches_service_public,90d12c615f,comment-est-calculee-lindemnite-de-conges-payes-du-salarie,Comment est calculée l'indemnité de congés payés du salarié ?,"pendant vos congés, vous ne percevez pas votre salaire. vous percevez toutefois une indemnité de congés payés. éléments de votre rémunération brute pris en compte ou non pris en compte dans le calcul de votre indemnité de congés payés tous les éléments de votre rémunération brute n'entrent pas en compte dans le calcul de votre indemnité de congés payés. sommes prises en compte et non prises en compte pour déterminer l'indemnité de vos congés payés somme prise en compte salaire de base oui majoration de salaire ( heures supplémentaires , travail de nuit , etc.) oui salaire reconstitué pendant des périodes assimilées à du travail effectif ( congé maternité , congé de paternité et d'accueil de l'enfant ou arrêt de travail pour accident de travail ou maladie professionnelle par exemple) oui indemnité de congés payés de l'année précédente oui prime d'ancienneté (si elle n'est pas versée pour l'année, périodes de travail et de congés confondues) oui prime d'assiduité versée mensuellement oui prime d'astreinte oui commissions pour les commerciaux oui prime d'expatriation oui avantages en nature oui prime de fin d'année non prime d'intéressement non prime de bilan non prime de participation non frais professionnels non 13 e mois (si la somme n'est pas versée pour l'année, périodes de travail et de congés confondues) oui activité partielle oui contrepartie financière au titre de l'obligation de non-concurrence oui indemnités de fin de contrat (dite prime de précarité) à durée déterminée oui indemnités de fin de mission de contrat d'intérim oui calcul de votre indemnité de congés payés selon 2 modes de calcul votre indemnité de congés payés est calculée par comparaison entre 2 modes de calcul : selon la 1 re méthode (dite du 1/10 e ), votre indemnité de congés payés est égale à 1/10 e de votre rémunération brute totale perçue au cours de votre période de référence selon la"
2,fiches_service_public,dd59ddc

,slug,cdtnId,exists,dp_pos,rr_pos
0,fiche-service-public/comment-est-calculee-lindemnite-de-conges-payes-du-salarie,90d12c615f,True,17,1
1,fiche-service-public/conges-payes,a5ca4ecbb9,True,1,10
2,fiche-service-public/comment-est-calculee-lindemnite-de-conges-payes-du-salarie,90d12c615f,True,17,1


In [11]:
pq = pd.read_csv("/home/remi/Downloads/xp-documents-prequalified.csv")

In [12]:
pq[:1]

,source,cdtnId,title,slug,text,answers.generic.text,refs.cdtnId,refs.slug,refs.source,variants
0,prequalified,10d6ccffb6,rupture-conventionnelle-refus,rupture-conventionnelle-refus,rupture-conventionnelle-refus,-,"36a5eef9e4, baf48a8630, 9934238010, 39abecf448...","rupture-conventionnelle, rupture-conventionnel...","fiches_service_public, page_fiche_ministere_tr...","rupture conventionnelle-refus, refus de ruptur..."


In [45]:
def toArray(flat):
    return flat.split(", ")
    
def parsePQ(row) :
    questions = toArray(row["variants"]) 
    cdtnIds = toArray(row["refs.cdtnId"])
    slugs = toArray(row["refs.slug"])
    sources = toArray(row["refs.source"])
    expectedResults = [{"cdtnId" : id, "slug": source+"/"+slug, "source":source} for [id, slug, source] in zip(cdtnIds, slugs, sources)]
    pqs = [{"question": question, "expectedResults": expectedResults} for question in questions[:3]]
    #print(pqs)
    return pqs

pqDF = pq[:15].apply(lambda row: parsePQ(row), axis=1).explode()
pqPrepro = pd.json_normalize(pqDF)
pqPrepro

,question,expectedResults
0,rupture conventionnelle-refus,"[{'cdtnId': '36a5eef9e4', 'slug': 'fiches_serv..."
1,refus de rupture conventionnelle,"[{'cdtnId': '36a5eef9e4', 'slug': 'fiches_serv..."
2,refus rupture conventionnelle,"[{'cdtnId': '36a5eef9e4', 'slug': 'fiches_serv..."
3,dpae,"[{'cdtnId': '799dc7e99a', 'slug': 'page_fiche_..."
4,déclaration préalable à l'embauche,"[{'cdtnId': '799dc7e99a', 'slug': 'page_fiche_..."
5,déclaration préalable,"[{'cdtnId': '799dc7e99a', 'slug': 'page_fiche_..."
6,obligation de loyauté,"[{'cdtnId': '978f53cb85', 'slug': 'fiches_serv..."
7,loyauté,"[{'cdtnId': '978f53cb85', 'slug': 'fiches_serv..."
8,CDI,"[{'cdtnId': 'c1120a94a5', 'slug': 'fiches_serv..."
9,contrat à durée indéterminée,"[{'cdtnId': 'c1120a94a5', 'slug': 'fiches_serv..."


In [46]:
for i in range(len(pqPrepro)) :    
    process(pqPrepro.iloc[i])
    print("\n")

0
rupture conventionnelle-refus


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiches_service_public/rupture-conventionnelle,36a5eef9e4,True,0,1
1,page_fiche_ministere_travail/rupture-conventionnelle-collective,baf48a8630,True,11,3
2,code_du_travail/l1237-11,9934238010,False,n/a,n/a
3,code_du_travail/l1237-13,39abecf448,False,n/a,n/a
4,code_du_travail/l1237-15,2aef0626d1,False,n/a,n/a
5,code_du_travail/l1237-14,c6d6ea86db,False,n/a,n/a
6,code_du_travail/l4733-9,baea1a8f10,False,n/a,n/a
7,code_du_travail/r1237-3,18ddc4a6e3,False,n/a,n/a




1
refus de rupture conventionnelle


,slug,cdtnId,exists,dp_pos,rr_pos
0,fiches_service_public/rupture-conventionnelle,36a5eef9e4,True,0,1
1,page_fiche_ministere_travail/rupture-conventionnelle-collective,baf48a8630,True,10,5
2,code_du_travail/l1237-11,9934238010,False,n/a,n/a
3,code_du_travail/l1237-13,39abecf448,False,n/a,n/a
4,code_du_travail/l1237-15,2aef0626d1,False,n/a,n/a
5,code_du_travail/l1237-14,c6d6ea86db,False,n/a,n/a
6,code_du_travail/l4733-9,baea1a8f10,False,n/a,n/a
7,code_du_travail/r1237-3,18ddc4a6e3,False,n/a,n/a




2
refus rupture conventionnelle


...

,slug,cdtnId,exists,dp_pos,rr_pos
0,information/rupture-conventionnelle-individuelle-la-procedure-en-details,188a439a07,False,n/a,n/a
1,fiches_service_public/rupture-conventionnelle,36a5eef9e4,True,0,0
2,page_fiche_ministere_travail/la-rupture-conventionnelle-du-contrat-de-travail-a-duree-indeterminee,1703b2338f,True,6,3
3,contributions/quest-ce-quune-rupture-conventionnelle,577163de72,True,7,2
4,modeles_de_courriers/demande-de-rendez-vous-en-vue-dune-rupture-conventionnelle,a712ecb976,False,n/a,n/a
5,external/telerc,f1064e4294,False,n/a,n/a
6,page_fiche_ministere_travail/rupture-conventionnelle-collective,baf48a8630,True,10,6
7,information/la-rupture-conventionnelle-est-elle-possible-pour-les-assistants-maternels-et-salaries-du-particulier-employeur,454426bcb2,False,n/a,n/a
8,code_du_travail/l1237-11,9934238010,False,n/a,n/a
9,code_du_travail/l1237-19-1,3fc6c60bef,False,n/a,n/a




40
licenciement rupture conventionnelle


,slug,cdtnId,exists,dp_pos,rr_pos
0,information/rupture-conventionnelle-individuelle-la-procedure-en-details,188a439a07,False,n/a,n/a
1,fiches_service_public/rupture-conventionnelle,36a5eef9e4,True,0,2
2,page_fiche_ministere_travail/la-rupture-conventionnelle-du-contrat-de-travail-a-duree-indeterminee,1703b2338f,True,3,1
3,contributions/quest-ce-quune-rupture-conventionnelle,577163de72,True,12,0
4,modeles_de_courriers/demande-de-rendez-vous-en-vue-dune-rupture-conventionnelle,a712ecb976,False,n/a,n/a
5,external/telerc,f1064e4294,False,n/a,n/a
6,page_fiche_ministere_travail/rupture-conventionnelle-collective,baf48a8630,True,18,7
7,information/la-rupture-conventionnelle-est-elle-possible-pour-les-assistants-maternels-et-salaries-du-particulier-employeur,454426bcb2,False,n/a,n/a
8,code_du_travail/l1237-11,9934238010,False,n/a,n/a
9,code_du_travail/l1237-19-1,3fc6c60bef,False,n/a,n/a




41
rupture conventionelle


,slug,cdtnId,exists,dp_pos,rr_pos
0,information/rupture-conventionnelle-individuelle-la-procedure-en-details,188a439a07,False,n/a,n/a
1,fiches_service_public/rupture-conventionnelle,36a5eef9e4,True,0,0
2,page_fiche_ministere_travail/la-rupture-conventionnelle-du-contrat-de-travail-a-duree-indeterminee,1703b2338f,True,6,5
3,contributions/quest-ce-quune-rupture-conventionnelle,577163de72,True,7,2
4,modeles_de_courriers/demande-de-rendez-vous-en-vue-dune-rupture-conventionnelle,a712ecb976,False,n/a,n/a
5,external/telerc,f1064e4294,False,n/a,n/a
6,page_fiche_ministere_travail/rupture-conventionnelle-collective,baf48a8630,True,8,6
7,information/la-rupture-conventionnelle-est-elle-possible-pour-les-assistants-maternels-et-salaries-du-particulier-employeur,454426bcb2,False,n/a,n/a
8,code_du_travail/l1237-11,9934238010,False,n/a,n/a
9,code_du_travail/l1237-19-1,3fc6c60bef,False,n/a,n/a
